In [1]:
import numpy as np
import pandas as pd
import boto3

In [2]:
# Let's use Amazon S3
# The Access Key ID and Secret Access Key are in ~/.aws/credentials
# Region is set in ~/.aws/config
s3 = boto3.resource('s3')

In [3]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

athena-query-bucket-testda24
aws-glue-assets-936480984004-eu-north-1
hacksprint-2023
temp-transformation-rs
vehicle-bucket-alarm


In [4]:
# Print the file names in the bucket
revolvos_bucket = s3.Bucket('hacksprint-2023')
for object in revolvos_bucket.objects.all():
    print(object.key)

Telematics Data Dictionary 2023_02_14.xlsx
Vehicle_Fault.csv
Vehicle_Fault_Alarm.csv
Vehicle_Fault_Alarm_Item.csv
Vehicle_Location.csv
Vehicle_Specs.csv


In [5]:
%%time
# Load one of the files into a dataframe
obj = s3.Object('hacksprint-2023', 'Vehicle_Location.csv').get()
df = pd.read_csv(obj['Body'])

CPU times: user 33.3 s, sys: 3.94 s, total: 37.3 s
Wall time: 1min 30s


In [6]:
df.columns

Index(['VEHICLE_ID', 'VEHICLE_MESSAGE_ID', 'DATE_TIME', 'LATITUDE',
       'LONGITUDE', 'CITY', 'STATE', 'ODOMETER_READING', 'ENGINE_STATUS',
       'ELEVATION', 'IDLE_ENGINE_HOURS', 'TOTAL_FUEL', 'GROSS_COMBINED_WEIGHT',
       'ENGINE_HOURS'],
      dtype='object')

In [7]:
print(df.dtypes)

VEHICLE_ID                 int64
VEHICLE_MESSAGE_ID         int64
DATE_TIME                 object
LATITUDE                 float64
LONGITUDE                float64
CITY                      object
STATE                     object
ODOMETER_READING         float64
ENGINE_STATUS              int64
ELEVATION                float64
IDLE_ENGINE_HOURS        float64
TOTAL_FUEL               float64
GROSS_COMBINED_WEIGHT    float64
ENGINE_HOURS             float64
dtype: object


In [8]:
print(df.head())

   VEHICLE_ID  VEHICLE_MESSAGE_ID             DATE_TIME   LATITUDE  \
0   100212831         10311092654  7/1/2022 12:05:23 AM  54.647507   
1   100212831         10311314269   7/1/2022 1:05:23 AM  54.647507   
2   100212831         10311519019   7/1/2022 2:05:23 AM  54.647507   
3   100212831         10311711700   7/1/2022 3:05:23 AM  54.652069   
4   100212831         10311892369   7/1/2022 4:05:23 AM  55.085544   

    LONGITUDE       CITY STATE  ODOMETER_READING  ENGINE_STATUS  ELEVATION  \
0 -118.983314        NaN   NaN        126151.836              1     3507.0   
1 -118.983314        NaN   NaN        126151.836              1     3507.0   
2 -118.983314        NaN   NaN        126151.836              1     3507.0   
3 -118.959104        NaN   NaN        126152.954              1     3412.0   
4 -118.810293  Grovedale    AB        126193.530              1     2152.0   

   IDLE_ENGINE_HOURS    TOTAL_FUEL  GROSS_COMBINED_WEIGHT  ENGINE_HOURS  
0           1944.503  38724.452811  

Insert magical cleaning...

In [ ]:
%%time
# Send the cleaned dataframe content back to a new CSV file on the S3 bucket
csv_clean = df.to_csv(index=False)
_ = s3.Object('hacksprint-2023', 'Vehicle_Location-clean.csv').put(Body=csv_clean)